In [67]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import os
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch import nn, optim
import torch.nn.functional as F
from numpy import asarray
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
import warnings; warnings.simplefilter('ignore')
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.nn import Module
%matplotlib inline
%config InlineBackend.figure_format='retina'

import torch.nn.functional as F
from torchvision.datasets.utils import download_url
device = ("cuda" if torch.cuda.is_available() else "cpu")

from google.colab import files

In [27]:
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
HAPPY_COLORS_PALETTE =\
["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#93D30C", "#8F00FF"]
sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))
rcParams['figure.figsize'] = 12, 8
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [2]:
uploaded = files.upload()

Saving airbnb_raw.csv to airbnb_raw.csv


In [3]:
data = pd.read_csv("airbnb_raw.csv")

In [4]:
def clean(df):
    df["Room type"] = df["Descriptions"].str.split(" in", expand=True)[0]
    df.loc[:, "Guests"] = df["Info"].str.rsplit('guest', 1).str.get(0)
    df.loc[:, "Bedrooms"] = df["Info"].str.split(" · ", n = 0, expand = True)[1].str.rsplit('bedroom', 1).str.get(0)
    df.loc[:, "Beds"] = df.loc[:, "Info"].str.split(" . ", n = 0, expand = True)[2].str.rsplit('bed', 1).str.get(0)
    df.loc[:, "Bathrooms"] = df.loc[:, "Info"].str.split(" . ", n = 0, expand = True)[3]
    df.loc[:,"Price"] = df.loc[:, "Prices"].str.split("$", n = 0, expand = True)[1].str.replace(" / night", "")
    df.loc[:,"Bathrooms"] = df["Bathrooms"].str.rsplit('shared', 1).str.get(0)
    df.loc[:,"Bathrooms"] = df["Bathrooms"].str.rsplit('bath', 1).str.get(0)
    df.loc[:,"Ratings"] = df.loc[:, "Ratings"].str.split(" ", n = 0, expand = True)[1].str.replace(" out of 5;", "")
    df.loc[:, "Reviews"] = df["Review number"].str.rsplit('reviews', 1).str.get(0)
    return df

In [5]:
def tonumeric(df):
    df.loc[:, "Guests"] = pd.to_numeric(df.Guests, errors = 'coerce')
    df.loc[:, "Beds"] = pd.to_numeric(df.Beds, errors = 'coerce')
    df.loc[:, "Bedrooms"] = pd.to_numeric(df.Bedrooms, errors = 'coerce')
    df.loc[:, "Bathrooms"] = pd.to_numeric(df.Bathrooms, errors = "coerce")
    df.loc[:, "Ratings"] = pd.to_numeric(df.Ratings, errors = 'coerce')
    df.loc[:, "Price"] = pd.to_numeric(df.Price, errors = 'coerce')
    df.loc[:, "Reviews"] = pd.to_numeric(df.Reviews, errors = 'coerce')    
    return df

In [6]:
df = clean(data)
df2 = tonumeric(df)

In [7]:
airbnb_data = df2.drop(["Unnamed: 0", "Prices", "Info", "Review number"], axis = 1)
airbnb_data = airbnb_data.dropna()

In [8]:
airbnb_data['Room type'].unique()

array(['Entire house', 'Private room', 'Hotel room', 'Entire apartment',
       'Shared room', 'Entire loft', 'Entire condominium',
       'Entire guest suite', 'Entire townhouse', 'Entire guesthouse',
       'Treehouse', 'Entire cottage', 'Tiny house', 'Camper/RV', 'Beds',
       'Boat', 'Room'], dtype=object)

In [9]:
a=['Hotel room', 'Camper/RV', 'Beds','Boat', 'Room', 'Treehouse']
airbnb_data = airbnb_data[~airbnb_data['Room type'].isin(a)]

In [10]:
airbnb_data.describe()

,Ratings,Guests,Bedrooms,Beds,Bathrooms,Price,Reviews
count,662.000000,662.000000,662.000000,662.000000,662.000000,662.000000,662.000000
mean,4.763489,2.688822,1.172205,1.480363,1.283233,79.507553,108.309668
std,0.269919,1.650149,0.464094,1.091325,0.786270,65.688250,122.365042
min,2.830000,1.000000,1.000000,0.000000,1.000000,15.000000,3.000000
25%,4.700000,2.000000,1.000000,1.000000,1.000000,39.000000,17.000000
50%,4.840000,2.000000,1.000000,1.000000,1.000000,61.000000,62.500000
75%,4.930000,4.000000,1.000000,2.000000,1.000000,99.000000,159.000000
max,5.000000,10.000000,4.000000,12.000000,10.000000,741.000000,667.000000


In [11]:
airbnb = airbnb_data[airbnb_data['Price'] <= 600]

In [12]:
#separating and encoding accommodation type
def label_accommodation(df):
  if df['Room type'] == 'Entire house':
    return 'Entire home/apt'
  if df['Room type'] == 'Entire apartment':
    return 'Entire home/apt'
  if df['Room type'] == 'Entire loft':
    return 'Entire home/apt'

  if df['Room type'] == 'Entire condominium':
    return 'Entire home/apt'
  if df['Room type'] == 'Entire guest suite':
    return 'Entire home/apt'
  if df['Room type'] == 'Entire townhouse':
    return 'Entire home/apt'

  if df['Room type'] == 'Entire guesthouse':
    return 'Entire home/apt'
  if df['Room type'] == 'Entire cottage':
    return 'Entire home/apt'
  if df['Room type'] == 'Tiny house':
    return 'Entire home/apt'

  if df['Room type'] == 'Private room':
    return 'Private room'
  if df['Room type'] == 'Shared room':
    return 'Shared room'

In [13]:
airbnb['Accommodation type'] = airbnb.apply (lambda df: label_accommodation(df), axis=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
airbnb['City'].replace({'New York': 1, 'Los Angeles': 2, 'San Francisco': 3, 'Boston': 4, 'Chicago': 5}, inplace = True)
airbnb['Accommodation type'].replace({'Entire home/apt': 0, 'Private room': 1, 'Shared room': 2}, inplace = True)

In [16]:
airbnb.loc[:, "City"] = pd.to_numeric(airbnb.City, errors = 'coerce')
airbnb.loc[:, "Accommodation type"] = pd.to_numeric(airbnb['Accommodation type'], errors = 'coerce')

In [17]:
airbnb.head()

,Names,Descriptions,Facilities,Ratings,Links,City,Room type,Guests,Bedrooms,Beds,Bathrooms,Price,Reviews,Accommodation type
2,Flatbush Hideaway - Quiet and close to subway!...,Entire house in Flatbush,Wifi·Selfcheck-in·Airconditioning·Kitchen,4.85,http://airbnb.com/rooms/31221559?previous_page...,1,Entire house,4,1.0,3.0,1.0,39.0,86.0,0
3,"Friendly, unpretentious, close to the subway -...",Private room in Park Slope,Wifi·Airconditioning·Kitchen,4.91,http://airbnb.com/rooms/23839200?previous_page...,1,Private room,2,1.0,1.0,1.0,39.0,80.0,1
7,Soho Private Clean & Quiet Bedroom - null - Ne...,Private room in Soho,Wifi·Kitchen,4.80,http://airbnb.com/rooms/35680818?previous_page...,1,Private room,1,1.0,1.0,1.0,39.0,10.0,1
10,"Gold Room in Old American Style House, Near Su...",Private room in Queens,Wifi·Indoorfireplace·Selfcheck-in·Airconditioning,5.00,http://airbnb.com/rooms/41986091?previous_page...,1,Private room,2,1.0,1.0,1.5,40.0,7.0,1
12,Downtown Bronx Studio 15 min ride to Manhattan...,Entire apartment in Melrose,Wifi·Selfcheck-in·Airconditioning·Kitchen,4.88,http://airbnb.com/rooms/35584546?previous_page...,1,Entire apartment,3,1.0,1.0,1.0,56.0,24.0,0


**Introduction**
In the previous project I have used the data from airbnb.com. In this project I will use Price as an ouput variable, and features like: Accommodation type, Number of Guests, Number of Bathrooms, Number of Beds, Number of Bedrooms, Number of Reviews, Ratings, and City. The reason for that choise is that the data has such an amount of features that determine the price.


In [22]:
X = airbnb[['Accommodation type', 'Guests', 'Bathrooms', 'Beds', 'Bedrooms', 'Reviews', 'Ratings', 'City']]
y = airbnb[['Price']]

In [23]:
scaler = MinMaxScaler()
# transform data
X = scaler.fit_transform(X.to_numpy())
print(X)

[[0.         0.33333333 0.         ... 0.125      0.93087558 0.        ]
 [0.5        0.11111111 0.         ... 0.11596386 0.95852535 0.        ]
 [0.5        0.         0.         ... 0.01054217 0.9078341  0.        ]
 ...
 [0.         0.11111111 0.         ... 0.41716867 0.99078341 0.5       ]
 [0.5        0.         0.         ... 0.45933735 0.89400922 0.5       ]
 [0.         0.33333333 0.         ... 0.         1.         0.5       ]]


In [24]:
inputs = torch.from_numpy(X).to(torch.float32)
targets = torch.from_numpy(y.to_numpy()).to(torch.float32)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [29]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(528, 8) (528, 1)
(133, 8) (133, 1)


In [32]:
dataset = TensorDataset(inputs, targets)

In [33]:
num_rows = len(airbnb)

In [35]:
val_p = 0.1 
val_size = int(num_rows * val_p)

train_size = num_rows - val_size

train_ds, val_ds = random_split(dataset,[train_size, val_size])
test_ds

In [36]:
print(len(train_ds))
print(len(val_ds))

595
66


In [37]:
batch_size = 4
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

In [ ]:
test_loader = DataLoader(dataset=test_ds, batch_size=1)

In [38]:
input_size = 8
output_size = 1

In [40]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size,output_size) 
        
    def forward(self, xb):
        out = self.linear(xb)                          
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 

        out = self(inputs)          

        loss = F.l1_loss(out, targets)                
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch

        out = self(inputs)

        loss = F.l1_loss(out, targets)                    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):

        if (epoch+1) % 50 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

In [41]:
model = MyModel()

In [42]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.2623,  0.0802, -0.3475,  0.1854,  0.1306,  0.0150,  0.1517,  0.0004]],
        requires_grad=True), Parameter containing:
 tensor([0.1956], requires_grad=True)]

In [43]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    res = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        res.append(result)
    return res

In [44]:
out = evaluate(model, val_loader)
print(out)

{'val_loss': 81.63219451904297}


In [45]:
epochs = 100
lr = 0.001
res1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [50], val_loss: 65.3948
Epoch [100], val_loss: 53.1638


In [46]:
epochs = 1000
lr = 0.05
res2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [50], val_loss: 29.2825
Epoch [100], val_loss: 28.2252
Epoch [150], val_loss: 28.1004
Epoch [200], val_loss: 28.0298
Epoch [250], val_loss: 28.0457
Epoch [300], val_loss: 28.0363
Epoch [350], val_loss: 28.0667
Epoch [400], val_loss: 28.0516
Epoch [450], val_loss: 28.1218
Epoch [500], val_loss: 28.1319
Epoch [550], val_loss: 28.1956
Epoch [600], val_loss: 28.1894
Epoch [650], val_loss: 28.1653
Epoch [700], val_loss: 28.1847
Epoch [750], val_loss: 28.1961
Epoch [800], val_loss: 28.1772
Epoch [850], val_loss: 28.1826
Epoch [900], val_loss: 28.1773
Epoch [950], val_loss: 28.1628
Epoch [1000], val_loss: 28.1741


In [47]:
epochs = 50
lr = 0.4
res3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [50], val_loss: 28.2151


In [51]:
epochs = 1000
lr = 0.001

In [54]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)               
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [55]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([0.0000, 0.3333, 0.0556, 0.2500, 0.0000, 0.1792, 0.9862, 0.7500])
Target: tensor([99.])
Prediction: tensor([96.7536])


In [62]:
input, target = val_ds[6]
predict_single(input, target, model)

Input: tensor([0.0000, 0.5556, 0.0000, 0.2500, 0.3333, 0.0482, 0.8664, 0.7500])
Target: tensor([275.])
Prediction: tensor([113.2658])


In [56]:
input, target = val_ds[8]
predict_single(input, target, model)

Input: tensor([0.0000, 0.2222, 0.0000, 0.1667, 0.0000, 0.0934, 0.9631, 0.2500])
Target: tensor([63.])
Prediction: tensor([89.8840])


In [57]:
input, target = val_ds[17]
predict_single(input, target, model)

Input: tensor([0.5000, 0.1111, 0.0000, 0.0833, 0.0000, 0.3765, 0.9217, 0.2500])
Target: tensor([43.])
Prediction: tensor([50.2127])


In [61]:
input, target = val_ds[21]
predict_single(input, target, model)

Input: tensor([0.5000, 0.0000, 0.0000, 0.0833, 0.0000, 0.0030, 0.7235, 0.0000])
Target: tensor([32.])
Prediction: tensor([34.4026])


As it could be seen this model predicts well the 'average' data, but the accuracy is small for outliers, too big or too little data, even though the data was scaled.

In another notebook the DNN model was performed for classification (accommodation types)